# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [11]:
%matplotlib widget

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key



## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [13]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,80.49,75,20,8.48,few clouds
1,1,Mataura,NZ,-46.1927,168.8643,66.13,43,18,7.36,few clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,-11.16,99,100,2.01,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,59.11,36,75,10.36,broken clouds
4,4,Nome,US,64.5011,-165.4064,5.49,99,94,4.52,overcast clouds
5,5,Kapaa,US,22.0752,-159.3190,80.92,70,0,8.05,clear sky
6,6,Sola,VU,-13.8833,167.5500,82.13,79,97,15.97,overcast clouds
7,7,Nishihara,JP,26.1842,127.7558,63.68,59,40,19.57,scattered clouds
8,8,Longyearbyen,SJ,78.2186,15.6401,26.44,86,75,10.36,broken clouds
9,9,Barranca,PE,-10.7500,-77.7667,70.11,72,100,5.66,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [14]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [15]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) | (city_data_df['Max Temp'] >= min_temp)]

# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,80.49,75,20,8.48,few clouds
1,1,Mataura,NZ,-46.1927,168.8643,66.13,43,18,7.36,few clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,-11.16,99,100,2.01,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,59.11,36,75,10.36,broken clouds
4,4,Nome,US,64.5011,-165.4064,5.49,99,94,4.52,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
676,676,Fenoarivo,MG,-18.9330,47.4354,64.47,82,75,3.44,broken clouds
677,677,Mbandaka,CD,0.0487,18.2603,73.18,73,100,2.46,overcast clouds
678,678,Okha,RU,53.5739,142.9478,-8.46,94,14,10.42,few clouds
679,679,Carinhanha,BR,-14.3047,-43.7650,70.20,90,100,1.12,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [16]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities =preferred_cities_df.dropna()
# Display sample data
clean_travel_cities.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [18]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,80.49,few clouds,-0.6000,73.0833,
1,Mataura,NZ,66.13,few clouds,-46.1927,168.8643,
2,Qaanaaq,GL,-11.16,overcast clouds,77.4840,-69.3632,
3,Punta Arenas,CL,59.11,broken clouds,-53.1500,-70.9167,
4,Nome,US,5.49,overcast clouds,64.5011,-165.4064,
...,...,...,...,...,...,...,...
676,Fenoarivo,MG,64.47,broken clouds,-18.9330,47.4354,
677,Mbandaka,CD,73.18,overcast clouds,0.0487,18.2603,
678,Okha,RU,-8.46,few clouds,53.5739,142.9478,
679,Carinhanha,BR,70.20,overcast clouds,-14.3047,-43.7650,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [19]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [20]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row['Lat'] 
    longitude = row['Lng'] 
    
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params) 
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"] 
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Hithadhoo - nearest hotel: Pebbles Inn
Mataura - nearest hotel: No hotel found
Qaanaaq - nearest hotel: Hotel Qaanaaq
Punta Arenas - nearest hotel: Hotel Lacolet
Nome - nearest hotel: No hotel found
Kapaa - nearest hotel: Pono Kai Resort
Sola - nearest hotel: No hotel found
Nishihara - nearest hotel: ユインチホテル南城
Longyearbyen - nearest hotel: The Vault
Barranca - nearest hotel: Emperador
Ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
Byron Bay - nearest hotel: Lord Byron Resort
Ushuaia - nearest hotel: Apart Hotel Aires del Beagle
Vila Franca Do Campo - nearest hotel: Vinha d'Areia Beach Hotel
Kodiak - nearest hotel: Shelikof Lodge
Cape Town - nearest hotel: Townhouse Hotel
Port Hardy - nearest hotel: No hotel found
San Rafael - nearest hotel: Hotel Regional
Samana - nearest hotel: Hotel Cotubanamá
Lebu - nearest hotel: No hotel found
Prien - nearest hotel: Golden Nugget Lake Charles Hotel & Casino
Miedzychod - nearest hotel: Neptun
Albany - nearest hot

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,80.49,few clouds,-0.6000,73.0833,Pebbles Inn
1,Mataura,NZ,66.13,few clouds,-46.1927,168.8643,No hotel found
2,Qaanaaq,GL,-11.16,overcast clouds,77.4840,-69.3632,Hotel Qaanaaq
3,Punta Arenas,CL,59.11,broken clouds,-53.1500,-70.9167,Hotel Lacolet
4,Nome,US,5.49,overcast clouds,64.5011,-165.4064,No hotel found
...,...,...,...,...,...,...,...
676,Fenoarivo,MG,64.47,broken clouds,-18.9330,47.4354,Les Herons
677,Mbandaka,CD,73.18,overcast clouds,0.0487,18.2603,Hôtel Epervier
678,Okha,RU,-8.46,few clouds,53.5739,142.9478,Сфера
679,Carinhanha,BR,70.20,overcast clouds,-14.3047,-43.7650,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [21]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == 'No hotel found'].index, inplace = True)
clean_hotel_df = hotel_df.dropna(axis="index", how="any") 
# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,80.49,few clouds,-0.6000,73.0833,Pebbles Inn
2,Qaanaaq,GL,-11.16,overcast clouds,77.4840,-69.3632,Hotel Qaanaaq
3,Punta Arenas,CL,59.11,broken clouds,-53.1500,-70.9167,Hotel Lacolet
5,Kapaa,US,80.92,clear sky,22.0752,-159.3190,Pono Kai Resort
7,Nishihara,JP,63.68,scattered clouds,26.1842,127.7558,ユインチホテル南城
8,Longyearbyen,SJ,26.44,broken clouds,78.2186,15.6401,The Vault
9,Barranca,PE,70.11,overcast clouds,-10.7500,-77.7667,Emperador
10,Ilulissat,GL,0.81,overcast clouds,69.2167,-51.1000,Best Western Plus Hotel Ilulissat
11,Byron Bay,AU,80.94,broken clouds,-28.6500,153.6167,Lord Byron Resort
12,Ushuaia,AR,56.86,scattered clouds,-54.8000,-68.3000,Apart Hotel Aires del Beagle


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [22]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [31]:
# Configure the map plot
map_plot_4 = clean_hotel_df.hvplot.points(
     "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600,
    size = "Max Temp",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_4



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)